In [ ]:
%matplotlib inline
import gzip
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import product

Get index pairs from stats file.

In [ ]:
tree = ET.parse('../output/Stats/DemultiplexingStats.xml')
root = tree.getroot()

index_pairs = []

for index_pair in root.iter('Barcode'):
  name = index_pair.get('name')
  if name != "all":
    index_pairs.append((name, int(index_pair[0].find('BarcodeCount').text)))

index_pairs = sorted(index_pairs, key=lambda x: x[1], reverse=True)
index_pairs_present = [index_pair for index_pair in index_pairs if index_pair[1]]

# index_pairs
# index_pairs_present

Load results from GSC qc run and compare to our demultiplexing run.

In [ ]:
GSC_results = pd.read_csv('../data/GSC_demultiplex_stats.csv')
# GSC_results

Check that our counts match the GSC counts for each index pair.

In [ ]:
comparison = ["{:<20} {:<10} {:<10} {:<10}".format(*['Barcode', 'Our Count', 'GSC Count', 'Equal'])]
for index_pair, count in index_pairs_present:
  if index_pair in list(GSC_results['Barcode sequence']):
    GSC_count = int((GSC_results.loc[GSC_results['Barcode sequence'] == index_pair].iloc[0]['PF Clusters']).replace(',', ''))
    comparison.append("{:<20} {:<10} {:<10} {:<10}".format(*[index_pair, count, GSC_count, count == GSC_count]))

print('\n'.join(comparison))

Only 'unknown' has different counts. This is because we gave them incorrect indexes for 9 of the libraries, so their reads were part of the 'unknown' counts when GSC first tried demultiplexing.

Get all barcodes that were not demultiplexed.

In [ ]:
with gzip.open('../output/Undetermined_S0_L001_R1_001.fastq.gz', 'rb') as f:
  lines = f.readlines()

discarded = []
for line in lines:
  line = line.decode('utf-8')
  if line[:7] == '@M00329':
    discarded.append(line[-18:-1])

print(len(discarded))

39915 reads didn't demultiplex. This matches the "unknown" count. A good portion of these discarded reads are PhiX pike-in, which had the index pair ATCTCGTA+TCTTTCCC.

In [ ]:
discarded_counts = Counter(discarded)
print(discarded_counts['ATCTCGTA+TCTTTCCC'])
discarded_unique = set(discarded)

11313 reads were from the PhiX with no mismatches in the barcode.

Remove PhiX index pair and any other index pairs from discarded_counts such that both indexes are within hamming distance of 1 from the true PhiX indexes (this simulates what bcl2fastq would do). Add the PhiX index and read counts to index_pairs_present. Also remove "unknown" from index_pairs_present.

In [ ]:
def hamming(a, b):
  return(sum(i != j for i, j in zip(a, b)))

correct_index1 = 'ATCTCGTA'
correct_index2 = 'TCTTTCCC'
s = 0
phiX_discarded = []
for discarded_pair in discarded_unique:
  index1 = discarded_pair[:8]
  index2 = discarded_pair[9:]
  if hamming(correct_index1, index1) <= 1 and hamming(correct_index2, index2) <= 1:
    s += discarded_counts[discarded_pair]
    phiX_discarded.append(discarded_pair)

print(len(index_pairs_present))
index_pairs_present = [i for i in index_pairs_present if i[0] != 'unknown']
print(len(index_pairs_present))
index_pairs_present.append(('ATCTCGTA+TCTTTCCC', s))

print(s)
print(len(phiX_discarded))

print(len(discarded_unique))
discarded_unique = [x for x in discarded_unique if x not in phiX_discarded]
print(len(discarded_unique))

print(len(discarded_counts))
for pair in phiX_discarded:
  discarded_counts.pop(pair)
print(len(discarded_counts))

Check how many unique index pairs are present in the discarded reads.

In [ ]:
print(len(discarded_counts))
# print(len(discarded_unique))

Check how many reads didn't demultiplex after removing PhiX reads.

In [ ]:
print(discarded_counts.total())
print(discarded_counts.total() + s)

28265 reads didn't demultiplex. Look at Distribution of read counts for index pairs that didn't demultiplex.

In [ ]:
plt.hist(discarded_counts.values(), bins=range(12))
plt.show()

In [ ]:
plt.hist(discarded_counts.values(), bins=range(3, 50))
plt.show()

Now check how many discarded reads had index pairs that were the result of a deletion in one or both indexes. For MiSeq, a deletion in either index will result in an 'A' at the 8th position of the index. For each correct index pair, generate all index pair combinations that could result from a single deletion in one or both indexes.

In [ ]:
def all_deletions(seq):
  deletions = [seq]
  for i in range(len(seq)):
    seq_bases = [base for base in seq]
    seq_bases.pop(i)
    seq_bases.append('A')
    deletions.append(''.join(seq_bases))
  return deletions

# print(all_deletions('ACGT'))

def all_deletion_combinations(idx1, idx2):
  return [i + '+' + j for i, j in product(all_deletions(idx1), all_deletions(idx2))]

# print(all_deletion_combinations('ACTG', 'GTCA'))

Get deletion index pairs present in the discarded reads and the counts for each pair.

In [ ]:
# barcodes_present.append(('ATCTCGTA+TCTTTCCC', 0)) # phiX

observed_deletions = {}

for index_pair, frequency in index_pairs_present:
  pair_observed_deletions = []
  deletions = all_deletion_combinations(index_pair[:8], index_pair[9:])
  for deletion in deletions:
    if deletion in discarded_unique:
      pair_observed_deletions.append((deletion, discarded_counts[deletion]))
  # Using Counter here accounts for cases where the same index pair is present
  # multiple times in the list of all possible combinations.
  observed_deletions[index_pair] = Counter(dict(pair_observed_deletions))

# observed_deletions['CTTCGGTC+CATCGTTA']


Check how many discarded reads were the result of a deletion in one or both indexes.

In [ ]:
print(sum(pairs.total() for pairs in observed_deletions.values()))

For each index pair, find all discarded index pairs such that each index in the discarded pair is within Levenshtein distance of two from the correct index.

In [ ]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [ ]:
similar_index_pairs = {}

for index_pair, frequency in index_pairs_present:
  similar_to_pair = []
  correct_index1 = index_pair[:8]
  correct_index2 = index_pair[9:]
  for discarded_pair in discarded_unique:
    index1 = discarded_pair[:8]
    index2 = discarded_pair[9:]
    if levenshtein(correct_index1, index1) <= 2 and levenshtein(correct_index2, index2) <= 2:
      similar_to_pair.append((discarded_pair, discarded_counts[discarded_pair]))
  similar_index_pairs[index_pair] = Counter(dict(similar_to_pair))

# similar_index_pairs['CTTCGGTC+CATCGTTA']

In [ ]:
print(sum(pairs.total() for pairs in similar_index_pairs.values()))

In [ ]:
index_stats = pd.DataFrame(index_pairs_present, columns=['Index', 'Demultiplexed Reads'])
index_stats = index_stats.drop(6) # get rid of unknown

In [ ]:
index_stats['Similar Index Count'] = [len(similar_index_pairs[index]) for index in index_stats['Index']]
index_stats['Similar Index Reads'] = [similar_index_pairs[index].total() for index in index_stats['Index']]
index_stats['Similar Index Percent'] = index_stats['Similar Index Reads']/index_stats['Demultiplexed Reads'] * 100

index_stats['Deletion Index Count'] = [len(observed_deletions[index]) for index in index_stats['Index']]
index_stats['Deletion Index Reads'] = [observed_deletions[index].total() for index in index_stats['Index']]
index_stats['Deletion Index Percent'] = index_stats['Deletion Index Reads']/index_stats['Demultiplexed Reads'] * 100
index_stats

The PhiX barcode has a much lower frequency of deletions compared to the other barcodes: ~0.02% vs ~0.2%.

In [ ]:
sum(index_stats['Similar Index Reads'])

12713 of the 28265 discarded reads had an index pair within our Levenshtein distance threshold of the correct index sequence.